---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [56]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
'''def plot_graph(G, weight_name=None):
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);'''
'''with open('Employee_Movie_Choices.txt', 'r') as f:
print(f.read())'''

"with open('Employee_Movie_Choices.txt', 'r') as f:\nprint(f.read())"

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [57]:
def answer_one():
        
    G_df = pd.read_csv('Employee_Movie_Choices.txt', delimiter='\t', skiprows = 1, names=['n1', 'n2'])
    G = nx.from_pandas_dataframe(G_df, 'n1', 'n2')
    return G
answer_one()

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [58]:
def answer_two():
    
    G = answer_one()
    for x in list(movies):
        G.add_node(x, {'type' : 'movie'})
    for y in list(employees):
        G.add_node(y, {'type' : 'employees'})
    
    
    return G
answer_two()

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [59]:
def answer_three():
        
    # Your Code Here
    
    return bipartite.weighted_projected_graph(answer_two(), employees)
answer_three()

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [76]:
def answer_four():
        
    G_bp = bipartite.weighted_projected_graph(answer_two(), employees)
    G1_df = pd.read_csv('Employee_Relationships.txt', delimiter='\t', header = None, names=['E1', 'E2'])
    G1_df.reset_index(inplace= True)
    G1_df.columns = ['E1','E2','Relation']
    L_edge = G_bp.edges(data=True)
    L_new = []
    for x in L_edge:
        L_new.append((x[1],x[0],x[2]))
    L_edge = L_edge+L_new
    df = pd.DataFrame(L_edge, columns=['E1', 'E2', 'Common'])
    df['Common'] = df['Common'].map(lambda x: x['weight'])
    df_merg = pd.merge(G1_df, df, how = 'left', on = ['E1','E2'])
    df_merg.fillna(0, inplace = True)
    df_merg = df_merg[['Relation','Common']]
    return df_merg.corr(method='pearson').iloc[0,1]#_merg
answer_four()

0.78839622217334759